In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [6]:
scales = [2]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    filter = (x < 50)
    filtered_x = s.where(filter, x)
    return s.median(filtered_x)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 2, param_scale: 2, scale_input_multiplier: 10) ------------->

+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0          | 0            | 0         | 0         | 0              | 0                | 0             | 0             | 0                  | 0                  | 0                      |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--

==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-304,280],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":5393,"total_const_size":1202,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709690386793}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.621981143951416 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 40.0
==== Generating Proof ====
proof:  {'instances': [['3e89c37e1116fe2ce68ab4e73ee44968d88afeb2eedb85f2d87c4d984264b311', '0100000000000000000000000000000000000000000000000000000000000000', 'a000000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x26e7e8be09c0f57035eac9e47b888c2f5dfd89611d86aa26d256ca3abde9d7721e7755c1b3c2cc287eae24e1bf8c3196f2e5ce4cb4417cd72fe5d2350c2fa80827b4aa02a35868f28b36aedfd3f7a0b7da33bc5cdd2344a7eb954e442699bab12015c6825408ef0e078ab05917433d3ab1248a6abc4b8b9cb071e6c90be76508150f50530685e761d90204579d9663d8fb398037155eec7be4da2f7fb2652a5a105d5f32b172a54e8440fa6a2da2b0a00f9286e5de59910e9a29740040820588185aa0784557a2663a0c2f6820942713e8bb650983058b9aa9dab9deb607a62b082912b958545e0ad1f31167a230c2ed15a37aaa7f32b201fa2916bcc0a743411363b3847772c9e4490d1f5430eea93e529ac12ea087d65878954ad021685e7c21ed15dc9b2bda2a

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [40.0]
